In [35]:
import os
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d
from sklearn.metrics import accuracy_score

# Load configuration file
cfg_file = "/home/iiitb/Downloads/randlanet_semantickitti.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

# Initialize model, dataset, and pipeline
model = ml3d.models.RandLANet(**cfg.model)
cfg.dataset['dataset_path'] = "/home/iiitb/Desktop/nuscenes/2DPASS/dataset/SemanticKitti2/SemanticKitti"
dataset = ml3d.datasets.SemanticKITTI(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

# Load pre-trained weights
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "randlanet_semantickitti_202201071330utc.pth"
randlanet_url = "https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth"
if not os.path.exists(ckpt_path):
    cmd = "wget {} -O {}".format(randlanet_url, ckpt_path)
    os.system(cmd)

# Load the parameters
pipeline.load_ckpt(ckpt_path=ckpt_path)

# Get the validation split
val_split = dataset.get_split("validation")

# Initialize lists to store ground truth labels and predicted labels
ground_truth_labels = []
predicted_labels = []
point_clouds = []

# Run inference on the first 50 samples
for i in range(10):
    data = val_split.get_data(i)
    result = pipeline.run_inference(data)
    ground_truth_labels.append(data['label'])
    predicted_labels.append(result['predict_labels'])
    point_clouds.append(data['point'])




test 0/1: 100%|████████████████████████| 88109/88109 [00:08<00:00, 10171.99it/s]

test 0/1: 100%|████████████████████████| 88029/88029 [00:08<00:00, 10311.09it/s]

test 0/1: 100%|████████████████████████| 87560/87560 [00:08<00:00, 10251.10it/s]

test 0/1: 100%|████████████████████████| 87578/87578 [00:08<00:00, 10317.47it/s]

test 0/1: 100%|█████████████████████████| 87571/87571 [00:11<00:00, 7934.79it/s]

test 0/1: 100%|████████████████████████| 87689/87689 [00:19<00:00, 33303.39it/s]

In [37]:
import open3d as o3d
import numpy as np

# Assuming `data` contains your point cloud data and `predict_labels` contains predicted labels

# Extracting point cloud coordinates from `data`
point_cloud = point_clouds[0]

# Extracting predicted labels
predicted_label = (predicted_labels[0]+1)%20
print(predicted_label)

# Creating an Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)

# Assigning random colors based on predicted labels
num_classes = 20
colors = np.random.rand(num_classes, 3)  # Generating random RGB colors for each class

# Map each class label to its corresponding random color
color_mapping = {label: colors[label] for label in np.unique(predicted_label)}

# Assigning colors to points based on predicted labels
assigned_colors = []
for label in predicted_label:
    assigned_colors.append(color_mapping[label])

pcd.colors = o3d.utility.Vector3dVector(assigned_colors)

# Visualizing the point cloud
o3d.visualization.draw_geometries([pcd])


[15 15 15 ...  9  9  9]


In [33]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

# Example data
ground_truth_labels_flat = np.concatenate(ground_truth_labels)
predicted_labels_flat = np.concatenate(predicted_labels)
predicted_labels_updated = (predicted_labels_flat + 1) % 20

# Accuracy
accuracy = accuracy_score(ground_truth_labels_flat, predicted_labels_updated)
print("Accuracy:", accuracy)

# Classification Report
class_report = classification_report(ground_truth_labels_flat, predicted_labels_updated)
print("Classification Report:\n", class_report)

# Visualization: Ground Truth vs Predicted Labels



Accuracy: 0.6939936409370362


/home/iiitb/anaconda3/envs/kitti/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/iiitb/anaconda3/envs/kitti/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/iiitb/anaconda3/envs/kitti/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00     81957
           1       0.86      0.97      0.92     20311
           2       0.00      0.00      0.00       209
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.86      0.28      0.42      2072
           7       0.00      0.00      0.00         1
           9       0.74      0.97      0.84    239421
          10       0.00      0.03      0.00      1172
          11       0.88      0.44      0.58    329825
          12       0.00      0.00      0.00         5
          13       0.37      0.55      0.44     26108
          14       0.20      0.69      0.31      7111
          15       0.82      0.86      0.84    214948
          16       0.65      0.71      0.68     38344
          17       0.77      0.84      0.80    255750
          18       0.24      0.80      0.37      5343
   

/home/iiitb/anaconda3/envs/kitti/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/iiitb/anaconda3/envs/kitti/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [34]:
from sklearn.metrics import confusion_matrix

import numpy as np

# Extract ground truth and predicted labels
ground_truth_labels = ground_truth_labels_flat  # Ground truth labels
predicted_labels = predicted_labels_updated  # Predicted labels

# Compute confusion matrix
conf_matrix = confusion_matrix(ground_truth_labels.flatten(), predicted_labels.flatten())

# Calculate intersection and union for each class
intersection = np.diag(conf_matrix)
ground_truth_count = conf_matrix.sum(axis=1)
predicted_count = conf_matrix.sum(axis=0)
union = ground_truth_count + predicted_count - intersection

# Calculate IoU for each class
iou_per_class = intersection / union

# Calculate mean IoU
mean_iou = np.mean(iou_per_class)

# Print IoU metrics
for class_idx, iou in enumerate(iou_per_class):
    print(f"IoU for class {class_idx}: {iou}")

print(f"Mean IoU: {mean_iou}")

            

IoU for class 0: 0.0
IoU for class 1: 0.8434017845707211
IoU for class 2: 0.0
IoU for class 3: 0.0
IoU for class 4: 0.0
IoU for class 5: 0.26786537574919317
IoU for class 6: 0.0
IoU for class 7: 0.724661152082918
IoU for class 8: 0.0005785461475632998
IoU for class 9: 0.4125731520276928
IoU for class 10: 0.0
IoU for class 11: 0.2808292998373538
IoU for class 12: 0.18019106182057756
IoU for class 13: 0.7223071983186409
IoU for class 14: 0.5107711990367068
IoU for class 15: 0.6713126572615985
IoU for class 16: 0.22397740349408934
IoU for class 17: 0.7856370322301536
Mean IoU: 0.3124503256987339


In [38]:
import os
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d
from sklearn.metrics import accuracy_score

# Load configuration file
cfg_file = "/home/iiitb/Downloads/kpconv_semantickitti.yml";
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

# Initialize model, dataset, and pipeline
model = ml3d.models.KPFCNN(**cfg.model);
cfg.dataset['dataset_path'] = "/home/iiitb/Desktop/nuscenes/2DPASS/dataset/SemanticKitti2/SemanticKitti"
dataset = ml3d.datasets.SemanticKITTI(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

# Load pre-trained weights
# ckpt_folder = "./logs/"
# os.makedirs(ckpt_folder, exist_ok=True)
# ckpt_path = ckpt_folder + "randlanet_semantickitti_202201071330utc.pth"
# randlanet_url = "https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth"
# if not os.path.exists(ckpt_path):
#     cmd = "wget {} -O {}".format(randlanet_url, ckpt_path)
#     os.system(cmd)
pipeline.load_ckpt(ckpt_path="/home/iiitb/Downloads/kpconv_semantickitti_202009090354utc.pth");

# Load the parameters

# Get the validation split
val_split = dataset.get_split("validation")

# Initialize lists to store ground truth labels and predicted labels
ground_truth_labels = []
predicted_labels = []
point_clouds = []

# Run inference on the first 50 samples
for i in range(10):
    data = val_split.get_data(i)
    result = pipeline.run_inference(data)
    ground_truth_labels.append(data['label'])
    predicted_labels.append(result['predict_labels'])
    point_clouds.append(data['point'])




test 0/1: 100%|█████████████████████████| 88109/88109 [00:21<00:00, 4182.13it/s]

test 0/1: 100%|█████████████████████████| 88029/88029 [00:20<00:00, 4251.98it/s]

test 0/1: 100%|█████████████████████████| 87560/87560 [00:20<00:00, 4214.05it/s]

test 0/1: 100%|█████████████████████████| 87578/87578 [00:20<00:00, 4199.82it/s]

test 0/1: 100%|█████████████████████████| 87571/87571 [00:20<00:00, 4286.57it/s]

test 0/1: 100%|█████████████████████████▉| 87621/87689 [00:19<00:00, 915.96it/s]

In [39]:
import open3d as o3d
import numpy as np

# Assuming `data` contains your point cloud data and `predict_labels` contains predicted labels

# Extracting point cloud coordinates from `data`
point_cloud = point_clouds[0]

# Extracting predicted labels
predicted_label = (predicted_labels[0]+1)%20
print(predicted_label)

# Creating an Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)

# Assigning random colors based on predicted labels
num_classes = 20
colors = np.random.rand(num_classes, 3)  # Generating random RGB colors for each class

# Map each class label to its corresponding random color
color_mapping = {label: colors[label] for label in np.unique(predicted_label)}

# Assigning colors to points based on predicted labels
assigned_colors = []
for label in predicted_label:
    assigned_colors.append(color_mapping[label])

pcd.colors = o3d.utility.Vector3dVector(assigned_colors)

# Visualizing the point cloud
o3d.visualization.draw_geometries([pcd])


[15 15 15 ... 13 13 13]



test 0/1: 100%|██████████████████████████| 87689/87689 [02:23<00:00, 915.96it/s]

In [40]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

# Example data
ground_truth_labels_flat = np.concatenate(ground_truth_labels)
predicted_labels_flat = np.concatenate(predicted_labels)
predicted_labels_updated = (predicted_labels_flat + 1) % 20

# Accuracy
accuracy = accuracy_score(ground_truth_labels_flat, predicted_labels_updated)
print("Accuracy:", accuracy)

# Classification Report
class_report = classification_report(ground_truth_labels_flat, predicted_labels_updated)
print("Classification Report:\n", class_report)

# Visualization: Ground Truth vs Predicted Labels



Accuracy: 0.7099489456231914


/home/iiitb/anaconda3/envs/kitti/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/iiitb/anaconda3/envs/kitti/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/iiitb/anaconda3/envs/kitti/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00     81957
           1       0.79      0.92      0.85     20311
           2       0.42      0.05      0.09       209
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.68      0.39      0.50      2072
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         0
           9       0.71      0.78      0.75    239421
          10       0.12      0.04      0.06      1172
          11       0.70      0.73      0.72    329825
          12       0.00      0.00      0.00         5
          13       0.28      0.72      0.40     26108
          14       0.44      0.45      0.45      7111
          15       0.71      0.98      0.82    214948
          16       0.85      0.63      0.73     38344
   

/home/iiitb/anaconda3/envs/kitti/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/iiitb/anaconda3/envs/kitti/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [41]:
from sklearn.metrics import confusion_matrix

import numpy as np

# Extract ground truth and predicted labels
ground_truth_labels = ground_truth_labels_flat  # Ground truth labels
predicted_labels = predicted_labels_updated  # Predicted labels

# Compute confusion matrix
conf_matrix = confusion_matrix(ground_truth_labels.flatten(), predicted_labels.flatten())

# Calculate intersection and union for each class
intersection = np.diag(conf_matrix)
ground_truth_count = conf_matrix.sum(axis=1)
predicted_count = conf_matrix.sum(axis=0)
union = ground_truth_count + predicted_count - intersection

# Calculate IoU for each class
iou_per_class = intersection / union

# Calculate mean IoU
mean_iou = np.mean(iou_per_class)

# Print IoU metrics
for class_idx, iou in enumerate(iou_per_class):
    print(f"IoU for class {class_idx}: {iou}")

print(f"Mean IoU: {mean_iou}")

            

IoU for class 0: 0.0
IoU for class 1: 0.7400580032577172
IoU for class 2: 0.049107142857142856
IoU for class 3: 0.0
IoU for class 4: 0.0
IoU for class 5: 0.0
IoU for class 6: 0.3314285714285714
IoU for class 7: 0.0
IoU for class 8: 0.0
IoU for class 9: 0.5940341352105684
IoU for class 10: 0.03143418467583497
IoU for class 11: 0.5565052628645866
IoU for class 12: 0.0
IoU for class 13: 0.2495105561140801
IoU for class 14: 0.2877717148224349
IoU for class 15: 0.6946707035622192
IoU for class 16: 0.5716403510836059
IoU for class 17: 0.6124276903867459
IoU for class 18: 0.18120212139068945
IoU for class 19: 0.09379744090219042
Mean IoU: 0.24967939392781938
